In [1]:
def run_rlt_model(X, y, preprocessor, task="regression", test_size=0.2, random_state=42):
    """
    Pipeline pour entraîner un modèle RLT (regression ou classification)
    et calculer les métriques.
    """
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state,
        stratify=y if task=="classification" else None
    )

    # Initialisation du modèle RLT (à remplacer par ton implémentation Python)
    if task=="regression":
        model = RLTRegressor(ntrees=100, nmin=2, muting_rate=0.5, k=2, alpha=0.25)
    else:
        model = RLTClassifier(ntrees=100, nmin=2, muting_rate=0.5, k=2, alpha=0.25)

    pipe = Pipeline([
        ("preprocessing", preprocessor),
        ("rlt", model)
    ])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    if task=="regression":
        from sklearn.metrics import mean_squared_error, r2_score
        mse = mean_squared_error(y_test, y_pred)
        rmse = mse ** 0.5
        r2 = r2_score(y_test, y_pred)
        metrics = {"RMSE": round(rmse, 4), "R2": round(r2,4)}
    else:
        from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
        try:
            y_prob = pipe.predict_proba(X_test)[:,1]
            auc = round(roc_auc_score(y_test, y_prob),4)
        except:
            auc = "N/A"
        metrics = {
            "Accuracy": round(accuracy_score(y_test, y_pred),4),
            "F1-score": round(f1_score(y_test, y_pred, average="weighted"),4),
            "ROC-AUC": auc
        }

    return metrics
